# Original LSCP

Author: Huanfa Chen, Rongbo Xu

In [1]:
# %time
import numpy
import geopandas 
import pandas
import pulp
from shapely.geometry import Point
import matplotlib.pyplot as plt
# from google.colab import files
import spopt
from spopt.locate.coverage import LSCP
import time

/opt/anaconda3/lib/python3.8/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


## Import data

In [2]:
# the service distance in metres (equal to 10 miles)
service_dist = 16093.4
# the distance greater than service distance
great_dist = 20000

In [4]:
# import distance data
# the distance between existing sites and covered MSOAs
df_distance_existing_sites_covered_MSOA = pandas.read_csv('../Data/distance_df_existing_sites_MSOA.csv')

# the distance between potential sites and uncovered MSOAs
df_distance_potential_sites_all_MSOA = pandas.read_csv('../Data/distance_df_potential_sites_all_MSOA.csv')

In [43]:
df_distance_existing_sites_covered_MSOA.head(10)

,Unnamed: 0,origin_id,dest_id,distance
0,122,E02002536,E122,6712.7
1,127,E02002536,E127,13881.2
2,137,E02002536,E137,13631.4
3,836,E02002536,E836,12395.3
4,838,E02002536,E838,12672.0
5,843,E02002536,E843,14449.4
6,844,E02002536,E844,14914.6
7,846,E02002536,E846,14449.4
8,849,E02002536,E849,14914.6
9,1722,E02002537,E122,7828.7


In [44]:
#总共多少个existing sites: 1600
df_distance_existing_sites_covered_MSOA.nunique()

Unnamed: 0    373286
origin_id       6408
dest_id         1600
distance      122263
dtype: int64

In [45]:
df_distance_potential_sites_all_MSOA.head(10)

,origin_id,dest_id,distance
0,E02002536,P14,7041.3
1,E02002536,P207,14950.7
2,E02002536,P354,2963.1
3,E02002537,P14,8157.3
4,E02002537,P207,16066.6
5,E02002537,P354,3153.8
6,E02002534,P123,13144.8
7,E02002534,P162,14661.7
8,E02002534,P207,12390.9
9,E02002535,P14,8752.8


In [46]:
#总共多少个potential sites: 21127
df_distance_potential_sites_all_MSOA.nunique()

origin_id      6788
dest_id       21127
distance     157528
dtype: int64

In [47]:
print(df_distance_existing_sites_covered_MSOA.columns)
print(df_distance_potential_sites_all_MSOA.columns)

Index(['Unnamed: 0', 'origin_id', 'dest_id', 'distance'], dtype='object')
Index(['origin_id', 'dest_id', 'distance'], dtype='object')


In [5]:
# combine two distance df
df_distance_existing_potential_sites_all_MSOAs = pandas.concat([df_distance_existing_sites_covered_MSOA, df_distance_potential_sites_all_MSOA], ignore_index=False)

In [49]:
df_distance_existing_potential_sites_all_MSOAs.head(10)

,Unnamed: 0,origin_id,dest_id,distance
0,122.0,E02002536,E122,6712.7
1,127.0,E02002536,E127,13881.2
2,137.0,E02002536,E137,13631.4
3,836.0,E02002536,E836,12395.3
4,838.0,E02002536,E838,12672.0
5,843.0,E02002536,E843,14449.4
6,844.0,E02002536,E844,14914.6
7,846.0,E02002536,E846,14449.4
8,849.0,E02002536,E849,14914.6
9,1722.0,E02002537,E122,7828.7


## Transform data

In [6]:
# transform the distance df to matrix
ntw_dist_piv = df_distance_existing_potential_sites_all_MSOAs.pivot_table(values="distance", index="origin_id", columns="dest_id")
# transform matrix into numpy array
cost_matrix = ntw_dist_piv.to_numpy()

In [51]:
ntw_dist_piv

dest_id,E0,E1,E10,E100,E1000,E1001,E1002,E1003,E1004,E1005,...,P999,P9991,P9992,P9993,P9994,P9995,P9996,P9997,P9998,P9999
origin_id,,,,,,,,,,,,,,,,,,,,,
E02000001,NaN,NaN,NaN,NaN,NaN,5747.1,3668.0,NaN,NaN,NaN,...,10097.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
E02000002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
E02000003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
E02000004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
E02000005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
E02006930,NaN,NaN,NaN,NaN,NaN,14784.0,6718.1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
E02006931,NaN,NaN,NaN,NaN,NaN,12446.5,8671.2,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
E02006932,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# save the column names as a list
list_site_ID = ntw_dist_piv.columns.to_list()

In [8]:
# if an element is NA or equal to or greater than the service distance in the array, it means it is greater than the predefined service distance. Set it as great_distance
cost_matrix[cost_matrix == service_dist] = great_dist
cost_matrix[numpy.isnan(cost_matrix)] = great_dist

In [54]:
cost_matrix

array([[20000., 20000., 20000., ..., 20000., 20000., 20000.],
       [20000., 20000., 20000., ..., 20000., 20000., 20000.],
       [20000., 20000., 20000., ..., 20000., 20000., 20000.],
       ...,
       [20000., 20000., 20000., ..., 20000., 20000., 20000.],
       [20000., 20000., 20000., ..., 20000., 20000., 20000.],
       [20000., 20000., 20000., ..., 20000., 20000., 20000.]])

In [55]:
cost_matrix.shape

(6788, 22727)

## 运行模型

In [9]:
# 设置参数
num_facilities = 22727
num_demand_points = 6788

In [10]:
import gurobipy as gp
from gurobipy import GRB

m = gp.Model('facility_location')
# 添加决策变量
select = m.addVars(num_facilities, vtype=GRB.BINARY, name='Select')
# 设置限制条件
    # 每个i在距离x(16093.4)内至少被1个j覆盖
m.addConstrs((gp.quicksum(select[j] for j in range(num_facilities) if cost_matrix[i,j] < 16093.4) >= 1  for i in range(num_demand_points)), name='Demand')

Set parameter Username
Academic license - for non-commercial use only - expires 2023-06-14


{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>,
 6: <gurobi.Constr *Awaiting Model Update*>,
 7: <gurobi.Constr *Awaiting Model Update*>,
 8: <gurobi.Constr *Awaiting Model Update*>,
 9: <gurobi.Constr *Awaiting Model Update*>,
 10: <gurobi.Constr *Awaiting Model Update*>,
 11: <gurobi.Constr *Awaiting Model Update*>,
 12: <gurobi.Constr *Awaiting Model Update*>,
 13: <gurobi.Constr *Awaiting Model Update*>,
 14: <gurobi.Constr *Awaiting Model Update*>,
 15: <gurobi.Constr *Awaiting Model Update*>,
 16: <gurobi.Constr *Awaiting Model Update*>,
 17: <gurobi.Constr *Awaiting Model Update*>,
 18: <gurobi.Constr *Awaiting Model Update*>,
 19: <gurobi.Constr *Awaiting Model Update*>,
 20: <gurobi.Constr *Awaiting Model Update*>,
 21: <gurobi.Constr *Awaiting Model Update*>

In [11]:
m.setObjective(gp.quicksum(select[j] for j in range(num_facilities)), GRB.MINIMIZE)
#m.setParam(GRB.Param.PoolSolutions, 5000)
#m.setParam(GRB.Param.PoolGap, 0)
#m.setParam(GRB.Param.PoolSearchMode, 2)
m.optimize()

Set parameter PoolSolutions to value 5000
Set parameter PoolGap to value 0
Set parameter PoolSearchMode to value 2
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[x86])
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 6788 rows, 22727 columns and 2611068 nonzeros
Model fingerprint: 0x11dd9b8f
Variable types: 0 continuous, 22727 integer (22727 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 426.0000000
Presolve removed 2191 rows and 1 columns
Presolve time: 4.13s
Presolved: 4597 rows, 22726 columns, 2335089 nonzeros
Variable types: 0 continuous, 22726 integer (22724 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
    1330    2.6253555e+02   8.249686e+03   0.000000e+00      5s
    8942    3.0632568e+02   0.000000e+00   0.000000e+00    